# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objectives

After completing this lab you will be able to:

*   Read CSV and JSON file types.
*   Extract data from the above file types.
*   Transform data.
*   Save the transformed data in a ready-to-load format which data engineers can use to load into an RDBMS.


Import the required modules and functions


In [ ]:
import glob                         # this module helps in selecting files
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

## Download Files


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2023-07-19 08:43:39--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2023-07-19 08:43:39 (331 MB/s) - ‘source.zip’ saved [2707/2707]



## Unzip Files


In [ ]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


## Set Paths


In [ ]:
tmpfile    = "temp.tmp"               # file used to store all extracted data
logfile    = "logfile.txt"            # all event logs will be stored in this file
targetfile = "transformed_data.csv"   # file where transformed data is stored

## Extract


### CSV Extract Function


In [ ]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

In [ ]:
extract_from_csv('/content/source3.csv')

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


In [ ]:
df1 = extract_from_csv('source1.csv')
df2 = extract_from_csv('source2.csv')
df3 = extract_from_csv('source3.csv')

In [ ]:
print(df1)
print(df2)
print(df3)

    name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30
    name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30
    name  height  weight
0   alex   65.78  112.99
1   ajay   71.52  136.49
2  alice   69.40  153.03
3   ravi   68.22  142.34
4    joe   67.79  144.30


### JSON Extract Function


In [ ]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

In [ ]:
dfjs1 = extract_from_json('/content/source1.json')
dfjs2 = extract_from_json('/content/source2.json')
dfjs3 = extract_from_json('/content/source3.json')

In [ ]:
print(dfjs1)
print(dfjs2)
print(dfjs3)

    name  height  weight
0   jack   68.70  123.30
1    tom   69.80  141.49
2  tracy   70.01  136.46
3   john   67.90  112.37
    name  height  weight
0   jack   68.70  123.30
1    tom   69.80  141.49
2  tracy   70.01  136.46
3   john   67.90  112.37
    name  height  weight
0   jack   68.70  123.30
1    tom   69.80  141.49
2  tracy   70.01  136.46
3   john   67.90  112.37


### XML Extract Function


In [ ]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = dataframe.append({"name":name, "height":height, "weight":weight}, ignore_index=True)
    return dataframe

In [ ]:
dfxml1 = extract_from_xml('/content/source1.xml')
dfxml2 = extract_from_xml('/content/source2.xml')
dfxml3 = extract_from_xml('/content/source3.xml')

In [ ]:
print(dfxml1)
print(dfxml2)
print(dfxml3)

    name  height  weight
0  simon   67.90  112.37
1  jacob   66.78  120.67
2  cindy   66.49  127.45
3   ivan   67.62  114.14
    name  height  weight
0  simon   67.90  112.37
1  jacob   66.78  120.67
2  cindy   66.49  127.45
3   ivan   67.62  114.14
    name  height  weight
0  simon   67.90  112.37
1  jacob   66.78  120.67
2  cindy   66.49  127.45
3   ivan   67.62  114.14


### Extract Function


In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['name','height','weight']) # create an empty data frame to hold extracted data

    #process all csv files
    for csvfile in glob.glob("*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)

    #process all json files
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)

    #process all xml files
    for xmlfile in glob.glob("*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)

    return extracted_data

In [ ]:
data = extract()

In [ ]:
data.head()

,name,height,weight
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30


## Transform


The transform function does the following tasks.

1.  Convert height which is in inches to millimeter
2.  Convert weight which is in pounds to kilograms


In [ ]:
def transform(data):
        #Convert height which is in inches to millimeter
        #Convert the datatype of the column into float
        #data.height = data.height.astype(float)
        #Convert inches to meters and round off to two decimals(one inch is 0.0254 meters)
        data['height'] = round(data.height * 0.0254,2)

        #Convert weight which is in pounds to kilograms
        #Convert the datatype of the column into float
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['weight'] = round(data.weight * 0.45359237,2)
        return data

In [ ]:
tx_data = transform(data)

In [ ]:
tx_data.head()

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45


## Loading


In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index=False)

In [ ]:
f = open("endfile.csv", "x")

In [ ]:
load('/content/endfile.csv',data)

In [ ]:
fin_data = pd.read_csv('/content/endfile.csv')
fin_data

## Logging


In [ ]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running ETL Process


In [ ]:
log("ETL Job Started")

In [ ]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

In [ ]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
...,...,...,...
73,ivan,1.72,51.77
74,simon,1.72,50.97
75,jacob,1.70,54.73
76,cindy,1.69,57.81


In [ ]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [ ]:
log("ETL Job Ended")

In [ ]:
file1 = open('/content/logfile.txt','r')

In [ ]:
file1.readlines()

["2023-Jul-19-10:47:20,It's a trial, calma tranquila \n",
 "2023-Jul-19-10:48:02,It's a trial, calma tranquila \n",
 '2023-Jul-19-10:48:13,ETL Job Started\n',
 '2023-Jul-19-10:48:17,Extract phase Started\n',
 '2023-Jul-19-10:48:17,Extract phase Ended\n',
 '2023-Jul-19-10:48:39,Transform phase Started\n',
 '2023-Jul-19-10:48:39,Transform phase Ended\n',
 '2023-Jul-19-10:48:47,Load phase Started\n',
 '2023-Jul-19-10:48:47,Load phase Ended\n',
 '2023-Jul-19-10:48:49,ETL Job Ended\n']

# Exercise


Using the example above complete the exercise below.


## Download Files


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2023-07-19 12:10:48--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2023-07-19 12:10:49 (1.09 GB/s) - ‘datasource.zip’ saved [4249/4249]



## Unzip Files


In [ ]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data


The file `dealership_data` contains CSV, JSON, and XML files for used car data which contain features named `car_model`, `year_of_manufacture`, `price`, and `fuel`.


## Set Paths


In [ ]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
newlogfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

## Extract


### Question 1: CSV Extract Function


In [ ]:
# Add the CSV extract function below


In [ ]:
def extract_csv(filepath):
  dataframe = pd.read_csv(filepath)
  return dataframe

In [ ]:
extract_csv('/content/dealership_data/used_car_prices1.csv')

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
5,vitara brezza,2018,13805.970149,Diesel
6,ciaz,2015,10074.626866,Petrol
7,s cross,2015,9701.492537,Diesel
8,ciaz,2016,13059.701493,Diesel
9,ciaz,2015,11119.402985,Diesel


### Question 2: JSON Extract Function


In [ ]:
# Add the JSON extract function below


In [ ]:
def extract_json(filepath):
  dataframe = pd.read_json(filepath, lines=True)
  return dataframe

In [ ]:
extract_json('/content/dealership_data/used_car_prices1.json')

,car_model,year_of_manufacture,price,fuel
0,ritz,2012,4626.865672,Diesel
1,ritz,2011,3507.462687,Petrol
2,swift,2014,7388.059701,Diesel
3,ertiga,2014,8955.223881,Diesel
4,dzire,2014,8208.955224,Diesel
5,sx4,2011,4402.985075,CNG
6,dzire,2015,6940.298507,Petrol
7,800,2003,522.388060,Petrol
8,alto k10,2016,4477.611940,Petrol
9,sx4,2003,3358.208955,Petrol


### Question 3: XML Extract Function


In [ ]:
# Add the XML extract function below, it is the same as the xml extract function above but the column names need to be renamed.


In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def extract_xml(filepath):
  # creating a dataframe to store our data
  dataframe = pd.DataFrame(columns = ['car_model','year_of_manufacture','price','fuel'])
  tree = ET.parse(filepath)
  # getting the parent root of the sml doc
  root = tree.getroot()
  for i in root:
    car_model = i.find('car_model').text
    year = i.find('year_of_manufacture').text
    price = float(i.find('price').text)
    fuel = i.find('fuel').text
    # adding to the dataframe
    dataframe = dataframe.append({'car_model':car_model,'year_of_manufacture':year,
                                  'price':price,'fuel':fuel},ignore_index=True)
    return dataframe




In [ ]:
data = extract_xml('/content/dealership_data/used_car_prices2.xml')

<ipython-input-128-b4fcd7fa248d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe = dataframe.append({'car_model':car_model,'year_of_manufacture':year,


In [ ]:
data

,car_model,year_of_manufacture,price,fuel
0,etios liva,2014,5895.522388,Diesel


### Question 4: Extract Function

Call the specific extract functions you created above by replacing the `ADD_FUNCTION_CALL` with the proper function call.


In [ ]:
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data

    #process all csv files
    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)

    #process all json files
    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index=True)

    #process all xml files
    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_xml(xmlfile), ignore_index=True)

    return extracted_data

In [ ]:
data = extract()

In [ ]:
data.head()

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel


## Transform


### Question 5: Transform

Round the `price` columns to 2 decimal places


In [ ]:
def transform(data):
  data['price'] = round(data['price'],2)
  return data

In [ ]:
transform(data)

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
58,ertiga,2015,8656.72,Petrol
59,ciaz,2017,11567.16,Petrol
60,corolla altis,2016,21985.07,Diesel
61,etios liva,2014,5895.52,Diesel


## Loading


### Question 6: Load


In [ ]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile, index=False)

In [ ]:
def load_cars(target_file,data):
  data.to_csv(target_file, index = False)

In [ ]:
load_cars(targetfile,data)

In [ ]:
pd.read_csv('/content/transformed_data.csv')

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel
...,...,...,...,...
58,ertiga,2015,8656.72,Petrol
59,ciaz,2017,11567.16,Petrol
60,corolla altis,2016,21985.07,Diesel
61,etios liva,2014,5895.52,Diesel


## Logging


### Question 7: Log

Make sure to change the name of the logfile to the one specified in the set paths section. Change the timestamp order to Hour-Minute-Second-Monthname-Day-Year.


In [ ]:
def log_sess(message):
  ts_format = '%H:%M:%S-%h-%d-%Y'
  now = datetime.now()
  ts = now.strftime(ts_format)
  with open('dealership_logfile.txt','a') as f:
    f.write(ts + ',' + message + '\n')

## Running ETL Process


### Question 8: ETL Process

Run all functions to extract, transform, and load the data. Make sure to log all events using the `log` function. Place your code under each comment.


In [ ]:
# Log that you have started the ETL process
log_sess('ETL job started...')
# Log that you have started the Extract step
log_sess('Extracation began')
# Call the Extract function
data = extract()
# Log that you have completed the Extract step
log_sess('Completed extraction process')
data

# Log that you have started the Transform step
log_sess('Transformation began...')
# Call the Transform function
tx_data = transform(data)
# Log that you have completed the Transform step
log_sess('Transformation complete')
tx_data

# Log that you have started the Load step
log_sess('Loading began...')
# Call the Load function
load_cars(targetfile,tx_data)
# Log that you have completed the Load step
log_sess('Loading complete')

# Log that you have completed the ETL process
log_sess('ETL completed')

In [ ]:
# confirming the progress from reports

In [ ]:
f1 = open('/content/dealership_logfile.txt','r')

In [ ]:
f1.readlines()

['13:50:47-Jul-19-2023,ETL job started.../n13:50:47-Jul-19-2023,Extracation began/n13:50:47-Jul-19-2023,Completed extraction process/n13:50:47-Jul-19-2023,Transformation began.../n13:50:47-Jul-19-2023,Transformation complete/n13:50:47-Jul-19-2023,Loading began.../n13:50:47-Jul-19-2023,Loading complete/n13:50:47-Jul-19-2023,ETL completed/n13:51:52-Jul-19-2023,ETL job started...\n',
 '13:51:52-Jul-19-2023,Extracation began\n',
 '13:51:52-Jul-19-2023,Completed extraction process\n',
 '13:51:52-Jul-19-2023,Transformation began...\n',
 '13:51:52-Jul-19-2023,Transformation complete\n',
 '13:51:52-Jul-19-2023,Loading began...\n',
 '13:51:52-Jul-19-2023,Loading complete\n',
 '13:51:52-Jul-19-2023,ETL completed\n']